In [ ]:
#Install required libraries
!pip install -U transformers datasets peft trl accelerate bitsandbytes
!pip install "unsloth @ git+https://github.com/unslothai/unsloth.git"
!pip install unsloth_zoo


In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("ShivomH/Mental-Health-Conversations")

# Shuffle and slice first 5000 rows 
dataset_small = dataset["train"].shuffle(seed=42).select(range(5000))


In [ ]:
def format_chat(example):
    return {
        "text": f"<|system|>\n{example['instruction']}\n<|user|>\n{example['input']}\n<|assistant|>\n{example['response']}"
    }

formatted_dataset = dataset_small.map(format_chat)


In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", 
    max_seq_length = 2048,
    dtype = "float16",
    load_in_4bit = True,
)


In [ ]:
from peft import PeftModel

if not isinstance(model, PeftModel):
    model = FastLanguageModel.get_peft_model(
        model,
        r = 8,
        lora_alpha = 16,
        lora_dropout = 0.0,
        target_modules = [
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj"
        ],
        bias = "none"
    )
else:
    print("LoRA already applied.")


In [ ]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=2048,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = formatted_dataset.map(tokenize, remove_columns=formatted_dataset.column_names)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "llama3-mentalhealth",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 2,
    num_train_epochs = 1,
    learning_rate = 2e-5,
    logging_steps = 10,
    save_steps = 100,
    fp16 = True,
    report_to = "none",
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer,
)

trainer.train()
